# 04 - Geração de Respostas via LLM

Este notebook implementa a **Etapa 4** do pipeline SINKT: enriquecimento das interações com respostas textuais e justificativas de erro geradas por LLM.

## Objetivo
Gerar respostas realistas e justificativas de erro usando LLM para as interações previamente simuladas.

## Entrada
- `data/output/notebooks/simulacao_interacoes/interactions_bkt.json`: Interações simuladas (sem respostas LLM)

## Saída
- `data/output/notebooks/geracao_respostas_llm/interactions_complete.json`: Interações completas com respostas e justificativas LLM

## Importação de Bibliotecas

In [ ]:
import json
import os
from datetime import datetime
from typing import Dict, List, Any
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import random
import time

try:
    from openai import OpenAI
    print("✅ OpenAI importado com sucesso")
except ImportError:
    print("❌ OpenAI não instalado. Instale com: pip install openai")
    raise

print("✅ Bibliotecas importadas com sucesso")

## Inicializar Cliente OpenAI

In [ ]:
client = OpenAI()

print("✅ Cliente OpenAI inicializado")
print("   Usando modelo: gpt-4o-mini")
print("   API Key: Carregada de OPENAI_API_KEY")

## Carregamento de Dados

In [ ]:
with open('data/output/notebooks/simulacao_interacoes/interactions_bkt.json', 'r', encoding='utf-8') as f:
    interactions_data = json.load(f)
interactions = interactions_data['interactions']

with open('data/json/questions_graph.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)
questions = {q['id']: q for q in questions_data.get('questions', [])}

print(f"✅ Dados carregados:")
print(f"  - Interações: {len(interactions)}")
print(f"  - Questões: {len(questions)}")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.7
LLM_MAX_TOKENS = 200
LLM_TIMEOUT = 10

CHECKPOINT_INTERVAL = 100
CHECKPOINT_FILE = 'data/output/notebooks/geracao_respostas_llm/interactions_llm_checkpoint.json'

print(f"✅ Configurações:")
print(f"  - Modelo LLM: {LLM_MODEL}")
print(f"  - Temperatura: {LLM_TEMPERATURE}")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} interações")

In [ ]:
LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.7
LLM_MAX_TOKENS = 200
LLM_TIMEOUT = 10

CHECKPOINT_INTERVAL = 100
CHECKPOINT_FILE = 'data/output/interactions_llm_checkpoint.json'

print(f"✅ Configurações:")
print(f"  - Modelo LLM: {LLM_MODEL}")
print(f"  - Temperatura: {LLM_TEMPERATURE}")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} interações")

## Funções de Geração com LLM

In [ ]:
def generate_response_with_llm(question: Dict, is_correct: bool, error_type: str = None) -> str:
    """Gera resposta realista usando OpenAI LLM."""
    question_text = question.get('q', 'Questão desconhecida')
    question_type = question.get('type', 'descriptive')
    
    if question_type == 'multiple_choice':
        if is_correct:
            correct_answer = question.get('ans', 'A')
            return f"Opção {correct_answer}"
        else:
            options = ['A', 'B', 'C', 'D']
            correct = question.get('ans', 'A')
            wrong_options = [o for o in options if o != correct]
            return f"Opção {random.choice(wrong_options)}"
    
    try:
        if is_correct:
            prompt = f"""Você é um estudante de Linux que entendeu bem o conceito. 
            Responda de forma clara e concisa (2-3 frases) a seguinte questão:
            
            {question_text}
            
            Responda como um estudante competente, sem ser muito formal."""
        else:
            error_descriptions = {
                'misconception': 'confundindo conceitos similares',
                'careless': 'cometendo um erro por descuido',
                'slip': 'cometendo um erro de digitação ou distração',
                'incomplete': 'dando uma resposta incompleta',
                'misunderstanding': 'entendendo mal o enunciado'
            }
            error_desc = error_descriptions.get(error_type, 'cometendo um erro')
            
            prompt = f"""Você é um estudante de Linux que tem dificuldades.
            Responda de forma realista (2-3 frases) a seguinte questão, {error_desc}:
            
            {question_text}
            
            Responda como um estudante que não entendeu completamente."""
        
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "Você é um estudante respondendo questões sobre Linux."},
                {"role": "user", "content": prompt}
            ],
            temperature=LLM_TEMPERATURE,
            max_tokens=LLM_MAX_TOKENS,
            timeout=LLM_TIMEOUT
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"  Erro ao chamar LLM: {str(e)}")
        return "[Resposta indisponível - LLM erro]"

def generate_error_justification_with_llm(question: Dict, student_response: str, 
                                         error_type: str, concept_name: str) -> str:
    """Gera justificativa de erro usando LLM para análise mais profunda."""
    question_text = question.get('q', 'Questão desconhecida')
    correct_answer = question.get('ans', 'N/A')
    
    try:
        prompt = f"""Analise o erro do estudante de forma concisa (1-2 frases).

Questão: {question_text}
Resposta Correta: {correct_answer}
Resposta do Estudante: {student_response}
Tipo de Erro: {error_type}
Conceito: {concept_name}

Forneça uma justificativa educacional breve sobre o erro cometido."""
        
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "Você é um tutor analisando erros de estudantes."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=150,
            timeout=LLM_TIMEOUT
        )
        
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Erro do tipo {error_type} relacionado ao conceito '{concept_name}'."

print("✅ Funções de geração LLM definidas")

output_file = 'data/output/notebooks/geracao_respostas_llm/interactions_complete.json'

In [ ]:
def save_checkpoint(enriched_interactions: List[Dict], last_idx: int):
    """Salva checkpoint incremental."""
    checkpoint_data = {
        "checkpoint_info": {
            "last_interaction_processed": last_idx,
            "total_processed": len(enriched_interactions),
            "timestamp": datetime.now().isoformat()
        },
        "interactions": enriched_interactions
    }
    
    with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)
    
    print(f"  Checkpoint salvo: {len(enriched_interactions)} interações processadas")

def load_checkpoint():
    """Carrega checkpoint se existir."""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data['interactions'], data['checkpoint_info']['last_interaction_processed']
    return [], -1

def enrich_interactions_with_llm(interactions: List[Dict], questions: Dict, 
                                resume: bool = True) -> List[Dict]:
    """Enriquece interações com respostas e justificativas LLM."""
    enriched_interactions = []
    start_idx = 0
    
    if resume:
        enriched_interactions, last_processed = load_checkpoint()
        if last_processed >= 0:
            start_idx = last_processed + 1
            print(f"Checkpoint encontrado! Retomando da interação {start_idx + 1}")
            print(f"   Interações já processadas: {len(enriched_interactions)}")
    
    total_interactions = len(interactions)
    
    for idx in range(start_idx, total_interactions):
        interaction = interactions[idx].copy()
        
        if (idx + 1) % 100 == 0 or idx == start_idx:
            print(f"  Processando interação {idx + 1}/{total_interactions}...")
        
        question_id = interaction['question_id']
        question = questions.get(question_id, {})
        
        is_correct = interaction['is_correct']
        error_type = interaction.get('error_type')
        
        response = generate_response_with_llm(question, is_correct, error_type)
        interaction['response'] = response
        
        if not is_correct and error_type:
            concept_name = interaction.get('concept_name', 'Conceito')
            justification = generate_error_justification_with_llm(
                question, response, error_type, concept_name
            )
            interaction['error_justification'] = justification
        else:
            interaction['error_justification'] = None
        
        enriched_interactions.append(interaction)
        
        if (idx + 1) % CHECKPOINT_INTERVAL == 0:
            save_checkpoint(enriched_interactions, idx)
    
    save_checkpoint(enriched_interactions, total_interactions - 1)
    
    return enriched_interactions

print("Iniciando enriquecimento com LLM...")
print(f"   Total de interações: {len(interactions)}")
print()

enriched_interactions = enrich_interactions_with_llm(interactions, questions, resume=True)

print(f"\n✅ {len(enriched_interactions)} interações enriquecidas com sucesso!")

## Salvamento das Interações Completas

In [ ]:
output_data = {
    "metadata": {
        "description": "Conjunto completo de interações com respostas e justificativas LLM",
        "version": "4.0.0",
        "created_at": datetime.now().isoformat(),
        "llm_model": LLM_MODEL,
        "llm_temperature": LLM_TEMPERATURE,
        "total_interactions": len(enriched_interactions)
    },
    "interactions": enriched_interactions
}

output_file = 'data/output/notebooks/geracao_respostas_llm/interactions_complete.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

if os.path.exists(CHECKPOINT_FILE):
    os.remove(CHECKPOINT_FILE)
    print(f"Checkpoint removido (processamento completo)")

print(f"\n✅ Interações completas salvas em: {output_file}")
print(f"Total de interações: {len(enriched_interactions)}")
print(f"Tamanho do arquivo: {os.path.getsize(output_file) / (1024*1024):.2f} MB")

## Análise de Métricas

### Qualidade das Respostas LLM

In [ ]:
def analyze_llm_response_quality(interactions: List[Dict]) -> Dict[str, Any]:
    """Analisa qualidade das respostas geradas por LLM."""
    total_responses = len(interactions)
    
    empty_responses = sum(1 for i in interactions if not i.get('response') or len(i.get('response', '').strip()) == 0)
    valid_responses = total_responses - empty_responses
    
    response_lengths = [len(i.get('response', '')) for i in interactions if i.get('response')]
    
    incorrect_with_justification = sum(
        1 for i in interactions 
        if not i['is_correct'] and i.get('error_justification')
    )
    
    total_incorrect = sum(1 for i in interactions if not i['is_correct'])
    
    return {
        'total_responses': total_responses,
        'valid_responses': valid_responses,
        'empty_responses': empty_responses,
        'validity_percentage': (valid_responses / total_responses * 100) if total_responses > 0 else 0,
        'response_length_stats': {
            'mean': np.mean(response_lengths) if response_lengths else 0,
            'median': np.median(response_lengths) if response_lengths else 0,
            'min': np.min(response_lengths) if response_lengths else 0,
            'max': np.max(response_lengths) if response_lengths else 0
        },
        'error_justifications': {
            'total_errors': total_incorrect,
            'with_justification': incorrect_with_justification,
            'coverage_percentage': (incorrect_with_justification / total_incorrect * 100) if total_incorrect > 0 else 0
        }
    }

llm_quality = analyze_llm_response_quality(enriched_interactions)

print("\n✅ Qualidade das Respostas LLM:\n")
print(f"  Total de Respostas: {llm_quality['total_responses']}")
print(f"  Respostas Válidas: {llm_quality['valid_responses']} ({llm_quality['validity_percentage']:.1f}%)")
print(f"  Respostas Vazias: {llm_quality['empty_responses']}")
print(f"\n  Tamanho das Respostas (caracteres):")
print(f"    Média: {llm_quality['response_length_stats']['mean']:.0f}")
print(f"    Mediana: {llm_quality['response_length_stats']['median']:.0f}")
print(f"    Range: [{llm_quality['response_length_stats']['min']:.0f}, {llm_quality['response_length_stats']['max']:.0f}]")
print(f"\n  Justificativas de Erro:")
print(f"    Total de Erros: {llm_quality['error_justifications']['total_errors']}")
print(f"    Com Justificativa: {llm_quality['error_justifications']['with_justification']} ({llm_quality['error_justifications']['coverage_percentage']:.1f}%)")

### Exemplos de Respostas e Justificativas

In [ ]:
print("\n✅ Exemplos de Interações Completas:\n")
print("="*80)

correct_examples = [i for i in enriched_interactions if i['is_correct']][:2]
incorrect_examples = [i for i in enriched_interactions if not i['is_correct']][:2]

print("\nEXEMPLOS DE RESPOSTAS CORRETAS:\n")
for i, interaction in enumerate(correct_examples, 1):
    print(f"Exemplo {i}:")
    print(f"  ID: {interaction['interaction_id']}")
    print(f"  Estudante: {interaction['student_id']}")
    print(f"  Conceito: {interaction.get('concept_name', 'N/A')}")
    print(f"  Resposta: {interaction['response'][:150]}..." if len(interaction['response']) > 150 else f"  Resposta: {interaction['response']}")
    print(f"  Domínio: {interaction['mastery_before']:.3f} -> {interaction['mastery_after']:.3f}")
    print("-" * 80)

print("\nEXEMPLOS DE RESPOSTAS INCORRETAS COM JUSTIFICATIVA:\n")
for i, interaction in enumerate(incorrect_examples, 1):
    print(f"Exemplo {i}:")
    print(f"  ID: {interaction['interaction_id']}")
    print(f"  Estudante: {interaction['student_id']}")
    print(f"  Conceito: {interaction.get('concept_name', 'N/A')}")
    print(f"  Tipo de Erro: {interaction.get('error_type', 'N/A')}")
    print(f"  Resposta: {interaction['response'][:150]}..." if len(interaction['response']) > 150 else f"  Resposta: {interaction['response']}")
    print(f"  Justificativa: {interaction.get('error_justification', 'N/A')}")
    print(f"  Domínio: {interaction['mastery_before']:.3f} -> {interaction['mastery_after']:.3f}")
    print("-" * 80)

### Análise de Insights das Justificativas

In [ ]:
def analyze_error_justification_insights(interactions: List[Dict]) -> Dict[str, Any]:
    """Analisa insights das justificativas de erro geradas por LLM."""
    error_justifications = [
        {
            'error_type': i['error_type'],
            'concept': i.get('concept_name', 'N/A'),
            'justification': i.get('error_justification', ''),
            'justification_length': len(i.get('error_justification', ''))
        }
        for i in interactions if not i['is_correct'] and i.get('error_justification')
    ]
    
    justification_lengths = [j['justification_length'] for j in error_justifications]
    
    insights_by_error_type = defaultdict(list)
    for j in error_justifications:
        insights_by_error_type[j['error_type']].append(j['justification_length'])
    
    error_type_stats = {}
    for error_type, lengths in insights_by_error_type.items():
        error_type_stats[error_type] = {
            'count': len(lengths),
            'avg_length': np.mean(lengths) if lengths else 0
        }
    
    return {
        'total_justifications': len(error_justifications),
        'justification_length_stats': {
            'mean': np.mean(justification_lengths) if justification_lengths else 0,
            'median': np.median(justification_lengths) if justification_lengths else 0,
            'min': np.min(justification_lengths) if justification_lengths else 0,
            'max': np.max(justification_lengths) if justification_lengths else 0
        },
        'by_error_type': error_type_stats
    }

justification_insights = analyze_error_justification_insights(enriched_interactions)

print("\n✅ Análise de Insights das Justificativas:\n")
print(f"  Total de Justificativas: {justification_insights['total_justifications']}")
print(f"\n  Tamanho das Justificativas (caracteres):")
print(f"    Média: {justification_insights['justification_length_stats']['mean']:.0f}")
print(f"    Mediana: {justification_insights['justification_length_stats']['median']:.0f}")
print(f"    Range: [{justification_insights['justification_length_stats']['min']:.0f}, {justification_insights['justification_length_stats']['max']:.0f}]")
print(f"\n  Justificativas por Tipo de Erro:")
for error_type, stats in justification_insights['by_error_type'].items():
    print(f"    - {error_type}: {stats['count']} justificativas (média {stats['avg_length']:.0f} chars)")

## Resumo da Execução

In [ ]:
print("\n" + "="*70)
print("✅ GERAÇÃO DE RESPOSTAS VIA LLM CONCLUÍDA COM SUCESSO!")
print("="*70)
print(f"\n  Arquivo gerado: {output_file}")
print(f"\n  Resumo:")
print(f"    - Total de interações: {len(enriched_interactions)}")
print(f"    - Respostas válidas: {llm_quality['validity_percentage']:.1f}%")
print(f"    - Justificativas de erro: {llm_quality['error_justifications']['coverage_percentage']:.1f}%")
print(f"    - Modelo LLM: {LLM_MODEL}")
print(f"\n✅ Pipeline de simulação completo!")
print("="*70)